In [1]:
import numpy as np
import pulp

# Example data
T = 30  # Planning horizon (days)
n = 3  # Number of stages in the production line
demand = np.random.randint(80, 120, T)  # Daily demand fluctuates
max_production = np.array([100, 90, 95])  # Maximum production capacity per stage

# Costs
holding_cost = 0.05  # Very low holding cost per unit per day to encourage minimal inventory
shortage_cost = 10.0  # High shortage cost per unit to avoid shortages

# Initialize the problem
prob = pulp.LpProblem("JIT_Lean_Manufacturing_Optimization", pulp.LpMinimize)

# Variables
production = pulp.LpVariable.dicts("production", ((i, t) for i in range(n) for t in range(T)), lowBound=0, cat='Continuous')
inventory = pulp.LpVariable.dicts("inventory", ((i, t) for i in range(n) for t in range(T)), lowBound=0, cat='Continuous')
shortage = pulp.LpVariable.dicts("shortage", (t for t in range(T)), lowBound=0, cat='Continuous')

# Objective function: Minimize total holding and shortage costs
prob += pulp.lpSum(inventory[i, t] * holding_cost + shortage[t] * shortage_cost for i in range(n) for t in range(T))

# Constraints
for t in range(T):
    # Ensure production meets demand exactly
    prob += pulp.lpSum(production[i, t] for i in range(n)) == demand[t]

    for i in range(n):
        if t == 0:
            # Assume zero initial inventory
            prob += inventory[i, t] == production[i, t] - demand[t] / n
        else:
            # Inventory balance equation with JIT approach
            prob += inventory[i, t] == inventory[i, t-1] + production[i, t] - demand[t] / n
        
        # Production capacity constraints for each stage
        prob += production[i, t] <= max_production[i]

        # Avoid holding inventory if possible
        prob += inventory[i, t] <= 5  # Allow minimal buffer

# Solve the problem
prob.solve()

# Print results
print("Status:", pulp.LpStatus[prob.status])
for t in range(T):
    print(f"Day {t+1}:")
    for i in range(n):
        print(f" Stage {i+1} Production: {production[i, t].varValue:.2f}")
    print(f" Inventory: {inventory[i, t].varValue:.2f}")
    print(f" Shortage: {shortage[t].varValue:.2f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/8f20c8d1926e4bbaa90f3aa316d51f72-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/8f20c8d1926e4bbaa90f3aa316d51f72-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 305 COLUMNS
At line 963 RHS
At line 1264 BOUNDS
At line 1265 ENDATA
Problem MODEL has 300 rows, 210 columns and 537 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-300) rows, 0 (-210) columns and 0 (-537) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -4.7920778e-11
After Postsolve, objective -4.7920778e-11, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -4.792077846e-11 - 0 iterations time 0.002, Presol